In [ ]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import matplotlib.font_manager as fm

colors = ["#00ADD8", "#dea584", "#89e051"]
sns.set_theme(font="Fira Sans", style="whitegrid", palette=sns.color_palette(colors))
# Load the JSON file
with open('../archive_extract/results/results.json', 'r') as f:
    data = json.load(f)


replacements = {
    'cha': 'ChaCha20Poly1305\n',
    'bla': 'Blake2,',
    'aes': 'Aes256Gcm\n',
    'sha': 'SHA3,',
    'non': 'None,',
    'str': 'Stream Mode',
    'blk': 'Block Mode',
    'gocryptfs': '',
    'default': 'None'
}

def rename_benchmark(name):
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name.replace('_', ' ')

def digest(name):
    if "bla" in name:
        return "Blake2"
    elif "sha" in name:
        return "SHA3"
    elif "non" in name:
        return "None"
    else:
        return ""

def mode(name):
    if "str" in name:
        return "Stream"
    elif "blk" in name:
        return "Block"
    else:
        return ""

def cipher(name):
    if "aes" in name:
        return "Aes256Gcm"
    elif "cha" in name:
        return "ChaCha20Poly1305"
    else:
        return ""

def my_f(name):
    if "gocryptfs" in name:
        return "gocryptfs"
    elif "default" in name:
        return "default"
    else:
        return "fscryptrs"


In [ ]:
benchmarks = [
    "cha_bla_blk", "cha_bla_str", "cha_sha_blk", "cha_sha_str", "cha_non_blk", "cha_non_str",
    "aes_bla_blk", "aes_bla_str", "aes_sha_blk", "aes_sha_str", "aes_non_blk", "aes_non_str",
    "default", "gocryptfs_aes", "gocryptfs_cha",
]

results_data = []
for item in data['results']:
    if item['command'] in benchmarks:
        results_data.append({
            'command': item['command'],
            'running_time': np.mean(item['times'])
        })

df = pd.DataFrame(results_data)

df['Filesystem'] = df['command'].apply(my_f)
df['renamed_command'] = df['command'].apply(rename_benchmark)

df_sorted = df.sort_values(by='running_time', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='running_time', y='renamed_command', data=df_sorted, hue="Filesystem")
plt.xlabel('Running Time (seconds)')
plt.ylabel('Benchmark')
plt.title('Benchmark Running Times (100x)')
plt.tight_layout()
#plt.savefig('benchmark_extract_all.svg')

In [ ]:
benchmarks = [
    "default", "gocryptfs_aes", "gocryptfs_cha", "aes_non_blk", "cha_non_blk", "aes_bla_blk", "cha_bla_blk",
]

results_data = []
for item in data['results']:
    if item['command'] in benchmarks:
        results_data.append({
            'command': item['command'],
            'running_time': np.mean(item['times'])
        })

df = pd.DataFrame(results_data)

df['Filesystem'] = df['command'].apply(my_f)
df['Digest'] = df['command'].apply(digest)
df['Mode'] = df['command'].apply(mode)
df['Cipher'] = df['command'].apply(cipher)
df['Types'] = df.Digest + ' ' + df.Mode
df['renamed_command'] = df['command'].apply(rename_benchmark)

df_sorted = df.sort_values(by='running_time', ascending=False)
x = df.Cipher + ' ' + df.Digest + ' ' + df.Mode

plt.figure(figsize=(12, 5))
sns.barplot(x='running_time', y='renamed_command', data=df_sorted, hue="Filesystem")
plt.xlabel('Running Time (seconds)')
plt.ylabel('')
plt.title('Extracting Linux Kernel (100 samples)')
plt.tight_layout()
#plt.savefig('benchmark_extract_other.svg')

In [ ]:
sns.set_theme(font="Fira Sans", style="whitegrid", palette="Paired")

benchmarks = [
    "cha_bla_blk", "cha_bla_str", "cha_sha_blk", "cha_sha_str", "cha_non_blk", "cha_non_str",
    "aes_bla_blk", "aes_bla_str", "aes_sha_blk", "aes_sha_str", "aes_non_blk", "aes_non_str",
]

results_data = []
for item in data['results']:
    if item['command'] in benchmarks:
        results_data.append({
            'command': item['command'],
            'running_time': np.mean(item['times'])
        })

df = pd.DataFrame(results_data)

df['Digest'] = df['command'].apply(digest)
df['Mode'] = df['command'].apply(mode)
df['Types'] = df.Digest + ' ' + df.Mode
df['renamed_command'] = df['command'].apply(rename_benchmark)

df_sorted = df.sort_values(by='running_time', ascending=False)

plt.figure(figsize=(12, 7))
sns.barplot(x='running_time', y='renamed_command', data=df_sorted, hue="Types")
plt.xlabel('Running Time (seconds)')
plt.ylabel('')
plt.title('Extracting Linux Kernel (100 samples)')
plt.tight_layout()
#plt.savefig('benchmark_extract_fscryptrs.svg')

In [ ]:
benchmarks = [
    "cha_bla_blk", "cha_bla_str", "cha_sha_blk", "cha_sha_str", "cha_non_blk", "cha_non_str",
    "aes_bla_blk", "aes_bla_str", "aes_sha_blk", "aes_sha_str", "aes_non_blk", "aes_non_str",
    "gocryptfs_aes", "gocryptfs_cha",
    # "default", 
]

results_data = []
for item in data['results']:
    if item['command'] in benchmarks:
        results_data.append({
            'command': item['command'],
            'running_time': np.mean(item['times'])
        })

df = pd.DataFrame(results_data)

baseline_command_mean = df['running_time'].min()

       
relative_df = df['running_time'].reset_index()
relative_df['relative_performance'] = df['running_time'] / baseline_command_mean
relative_df = relative_df.sort_values(by='relative_performance', ascending=True)
display(relative_df)
